In [ ]:
!pip install -U pynwb
from pynwb import NWBHDF5IO
!pip install nwbwidgets
from nwbwidgets import nwb2widget
from scipy.signal import butter, filtfilt
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import random

In [ ]:
# opening ecephys probe file
!ls
filepath = '/Users/poggi/Documents/Maier Lab/NWB Data/sub-699733573_ses-715093703_probe-810755797_ecephys.nwb'
io = NWBHDF5IO(filepath, 'r',load_namespaces = True)  # open the file in read mode 'r'
nwb = io.read() # nwb dataset

In [ ]:
lfp_data = nwb.acquisition
lfp_data = lfp_data['probe_810755797_lfp_data']
lfp_data = lfp_data.data
lfp_df = pd.DataFrame(lfp_data)
shape = lfp_df.shape
print(shape)
# Retrieve the sampling rate from the NWB file
sampling_rate = nwb.electrode_groups['probeA'].lfp_sampling_rate

In [ ]:
# Convert the desired time range in seconds to indices
start_time_seconds = 0
end_time_seconds = 250  # 250 seconds
start_index = int(start_time_seconds * sampling_rate)
end_index = int(end_time_seconds * sampling_rate)

# Slice the LFP data
lfp_data_sliced = lfp_df.iloc[start_index:end_index]

# Select three random channels
num_channels = 3
all_channels = lfp_data_sliced.columns.tolist()
random_channels = random.sample(all_channels, num_channels)

# Slice the data for the selected random channels
lfp_data_sliced_random = lfp_data_sliced[random_channels]

In [ ]:
# graph the three channels
fig, ax = plt.subplots(num_channels, 1, figsize=(10, 10), sharex=True)
for i in range(num_channels):
    ax[i].plot(lfp_data_sliced_random.iloc[:, i])
    ax[i].set_ylabel('Channel {}'.format(random_channels[i]))
    ax[i].set_xlim([start_index, end_index])
    ax[i].set_ylim([-.0005, .0005])
    ax[i].set_yticks([-.0005, 0, .0005])
    ax[i].set_yticklabels([-.0005, 0, .0005])
    ax[i].grid(True)
ax[i].set_xlabel('Time (samples)')
plt.show()

In [ ]:
from scipy.signal import butter, filtfilt
import numpy as np

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    y_abs = np.abs(y)
    return y_abs

# Retrieve the sampling rate from the NWB file
sampling_rate = nwb.electrode_groups['probeA'].lfp_sampling_rate

# Apply bandpass filter to each of the random channels separately
filtered_lfp_sliced_random = lfp_data_sliced_random.apply(lambda x: bandpass_filter(x, lowcut=8, highcut=12, fs=sampling_rate))


In [ ]:
# graph the three channels
fig, ax = plt.subplots(num_channels, 1, figsize=(10, 10), sharex=True)
for i in range(num_channels):
    ax[i].plot(filtered_lfp_sliced_random.iloc[:, i])
    ax[i].set_ylabel('Channel {}'.format(random_channels[i]))
    ax[i].set_xlim([start_index, end_index])
    ax[i].set_ylim([0, .00005])
    ax[i].set_yticks([0, .0001, .0003])
    ax[i].set_yticklabels([0, .0001, .0003])
    ax[i].grid(True)

In [ ]:
# Apply bandpass filter to each of the random channels separately
filtered_lfp_sliced_random = lfp_data_sliced_random.apply(lambda x: bandpass_filter(x, lowcut=35, highcut=65, fs=sampling_rate))


In [ ]:
# graph the three channels
fig, ax = plt.subplots(num_channels, 1, figsize=(10, 10), sharex=True)
for i in range(num_channels):
    ax[i].plot(filtered_lfp_sliced_random.iloc[:, i])
    ax[i].set_ylabel('Channel {}'.format(random_channels[i]))
    ax[i].set_xlim([start_index, end_index])
    ax[i].set_ylim([-.0001, .0001])
    ax[i].set_yticks([-.001, 0, .001])
    ax[i].set_yticklabels([-.001, 0, .001])
    ax[i].grid(True)